# 🎬 IMDb Movie Chatbot - Enhanced Multi-Agent System

## Google Colab Version

### Features:
- **10+ Specialized AI Agents** for different query types
- **Mood-Based Recommendations** - Movies based on how you feel
- **Movie Trivia & Facts** - Behind-the-scenes knowledge
- **Review/Sentiment Analysis** - Critical reception insights
- **Advanced FAISS Vector Search** - Fast, accurate retrieval
- **Intelligent Caching** - Faster repeated queries
- **Conversation Memory** - Context retention
- **Comprehensive Error Handling**

---

In [ ]:
# Cell 1: Install Dependencies (Run this first!)
# This will take 1-2 minutes

# Install required packages (ignore dependency warnings - they don't affect functionality)
!pip install -q --upgrade langchain langchain-openai langchain-community langchain-text-splitters faiss-cpu gradio python-dotenv openai 2>/dev/null

# Verify installations
import importlib
packages = ['langchain', 'langchain_openai', 'faiss', 'gradio', 'openai']
all_ok = True
for pkg in packages:
    try:
        importlib.import_module(pkg.replace('-', '_'))
    except ImportError:
        all_ok = False
        print(f"❌ Failed to import {pkg}")

if all_ok:
    print("✅ All dependencies installed successfully!")
else:
    print("⚠️ Some packages may need manual installation")

In [ ]:
# Cell 2: Mount Google Drive & Set Dataset Path
# Upload your IMDb_Dataset.csv to Google Drive first

from google.colab import drive
drive.mount('/content/drive')

# UPDATE THIS PATH to match your Google Drive location
DATASET_PATH = '/content/drive/MyDrive/IMDb_Dataset (1).csv'

# Alternative: Upload directly to Colab (uncomment below)
# from google.colab import files
# uploaded = files.upload()
# DATASET_PATH = list(uploaded.keys())[0]

print(f"Dataset path set to: {DATASET_PATH}")

In [ ]:
# Cell 3: Set OpenAI API Key

import os

# Option A: Try to get from Colab Secrets first
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab Secrets")
except:
    OPENAI_API_KEY = None

# Option B: If not in secrets, prompt for input
if not OPENAI_API_KEY:
    from getpass import getpass
    OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")
    print("✅ API key entered manually")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ OpenAI API key configured!")

In [ ]:
# Cell 4: Import All Libraries

import os
import logging
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Tuple, List, Dict, Optional, Any, Set
import time
import random
import hashlib
import re
from collections import deque, OrderedDict
from abc import ABC, abstractmethod

# LangChain imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Gradio for UI
import gradio as gr

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Logging setup
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
log_filename = os.path.join(LOG_DIR, f"chatbot_{datetime.now().strftime('%Y%m%d')}.log")

logger = logging.getLogger("MovieChatbot")
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(handler)

print("✅ All libraries imported successfully!")

In [ ]:
# Cell 5: Load and Explore Dataset

df = pd.read_csv(DATASET_PATH)

print(f"✅ Dataset loaded: {df.shape[0]} movies, {df.shape[1]} features")
print(f"\nColumns: {list(df.columns)}")

# Statistics
print(f"\n📊 Dataset Statistics:")
print(f"   Year Range: {df['Year'].min()} - {df['Year'].max()}")
print(f"   Rating Range: {df['IMDb Rating'].min()} - {df['IMDb Rating'].max()}")
print(f"   Unique Directors: {df['Director'].nunique()}")
print(f"   Unique Genres: {df['Genre'].nunique()}")

# Display sample
df.head()

In [ ]:
# Cell 6: Create Movie Descriptions

def create_movie_description(row):
    """Create a rich text description for each movie."""
    title = row['Title'] if pd.notna(row['Title']) else 'Unknown Title'
    year = int(row['Year']) if pd.notna(row['Year']) else 'Unknown Year'
    genre = row['Genre'] if pd.notna(row['Genre']) else 'Unknown Genre'
    director = row['Director'] if pd.notna(row['Director']) else 'Unknown Director'
    cast = row['Star Cast'] if pd.notna(row['Star Cast']) else 'Unknown Cast'
    rating = row['IMDb Rating'] if pd.notna(row['IMDb Rating']) else 'N/A'
    metascore = row['MetaScore'] if pd.notna(row['MetaScore']) else 'N/A'
    certificate = row['Certificates'] if pd.notna(row['Certificates']) else 'Not Rated'
    duration = int(row['Duration (minutes)']) if pd.notna(row['Duration (minutes)']) else 'Unknown'
    poster = row['Poster-src'] if pd.notna(row['Poster-src']) else ''

    description = f"""Movie Title: {title}
Year: {year}
Genre: {genre}
Director: {director}
Star Cast: {cast}
IMDb Rating: {rating}/10
MetaScore: {metascore}
Certificate: {certificate}
Duration: {duration} minutes
Poster URL: {poster}

This is a {genre} movie titled "{title}" released in {year}, directed by {director} and starring {cast}. It has an IMDb rating of {rating}/10 and MetaScore of {metascore}. The film runs for {duration} minutes."""

    return description

print("Creating movie descriptions...")
df['description'] = df.apply(create_movie_description, axis=1)

print(f"\n✅ Created descriptions for {len(df)} movies")
print(f"\nSample description:")
print("-" * 50)
print(df['description'].iloc[0])

In [ ]:
# Cell 7: Create Documents and Embeddings

def create_documents_from_dataframe(df):
    """Convert DataFrame rows to LangChain Document objects."""
    documents = []

    for idx, row in df.iterrows():
        metadata = {
            'title': row['Title'] if pd.notna(row['Title']) else 'Unknown',
            'year': int(row['Year']) if pd.notna(row['Year']) else 0,
            'genre': row['Genre'] if pd.notna(row['Genre']) else 'Unknown',
            'director': row['Director'] if pd.notna(row['Director']) else 'Unknown',
            'rating': float(row['IMDb Rating']) if pd.notna(row['IMDb Rating']) else 0.0,
            'metascore': float(row['MetaScore']) if pd.notna(row['MetaScore']) else 0.0,
            'certificate': row['Certificates'] if pd.notna(row['Certificates']) else 'Not Rated',
            'poster_url': row['Poster-src'] if pd.notna(row['Poster-src']) else '',
            'duration': int(row['Duration (minutes)']) if pd.notna(row['Duration (minutes)']) else 0,
            'cast': row['Star Cast'] if pd.notna(row['Star Cast']) else 'Unknown',
        }

        doc = Document(page_content=row['description'], metadata=metadata)
        documents.append(doc)

    return documents

print("Converting to documents...")
documents = create_documents_from_dataframe(df)
print(f"\n✅ Created {len(documents)} documents")

In [ ]:
# Cell 8: Create FAISS Vector Store

print("Initializing embeddings model...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("Creating FAISS vector store...")
vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

print(f"\n✅ Vector store created with {vectorstore.index.ntotal} vectors")

# Test similarity search
print("\nTesting similarity search...")
test_results = vectorstore.similarity_search("comedy movie", k=3)
for i, doc in enumerate(test_results, 1):
    print(f"{i}. {doc.metadata['title']} ({doc.metadata['year']}) - {doc.metadata['rating']}/10")

In [ ]:
# Cell 9: Initialize LLM

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=1200,
)

print("✅ LLM initialized")
print("   Model: gpt-4o-mini")
print("   Temperature: 0.7")

# Test LLM
test_response = llm.invoke("Say 'Hello, movie fan!' in one sentence.")
print(f"\n🤖 LLM Test: {test_response.content}")

In [ ]:
# Cell 10: Configuration & Constants

# Cache configuration
CACHE_MAX_SIZE = 100
CACHE_TTL_SECONDS = 3600  # 1 hour
SEMANTIC_SIMILARITY_THRESHOLD = 0.92

# Rate limiting
RATE_LIMIT_REQUESTS = 30
RATE_LIMIT_WINDOW = 60  # seconds

# Mood to Genre Mapping for Mood-Based Recommendations
MOOD_GENRE_MAPPING = {
    "happy": ["Comedy", "Animation", "Musical", "Family"],
    "sad": ["Drama", "Romance"],
    "excited": ["Action", "Adventure", "Sci-Fi", "Thriller"],
    "scared": ["Horror", "Mystery", "Thriller"],
    "romantic": ["Romance", "Drama", "Comedy"],
    "thoughtful": ["Documentary", "Biography", "Drama", "History"],
    "nostalgic": ["Classic", "Family", "Animation"],
    "adventurous": ["Adventure", "Action", "Fantasy", "Sci-Fi"],
    "relaxed": ["Comedy", "Animation", "Family", "Documentary"],
    "inspired": ["Biography", "Documentary", "Drama", "Sport"]
}

print("✅ Configuration loaded")
print(f"   Cache Size: {CACHE_MAX_SIZE}")
print(f"   Rate Limit: {RATE_LIMIT_REQUESTS} requests per {RATE_LIMIT_WINDOW}s")
print(f"   Mood Categories: {len(MOOD_GENRE_MAPPING)}")

In [ ]:
# Cell 11: Caching and Rate Limiting

class QueryCache:
    """Dual-layer caching with exact match and semantic similarity."""

    def __init__(self, max_size: int = CACHE_MAX_SIZE, ttl: int = CACHE_TTL_SECONDS):
        self.max_size = max_size
        self.ttl = ttl
        self.exact_cache: OrderedDict = OrderedDict()
        self.stats = {"exact_hits": 0, "misses": 0}

    def _hash_query(self, query: str) -> str:
        return hashlib.md5(query.lower().strip().encode()).hexdigest()

    def get(self, query: str) -> Optional[Dict]:
        query_hash = self._hash_query(query)
        if query_hash in self.exact_cache:
            entry = self.exact_cache[query_hash]
            if time.time() - entry['timestamp'] < self.ttl:
                self.stats["exact_hits"] += 1
                return entry['response']
            del self.exact_cache[query_hash]
        self.stats["misses"] += 1
        return None

    def set(self, query: str, response: Dict):
        while len(self.exact_cache) >= self.max_size:
            self.exact_cache.popitem(last=False)
        query_hash = self._hash_query(query)
        self.exact_cache[query_hash] = {'response': response, 'timestamp': time.time()}

    def get_stats(self) -> Dict:
        total = sum(self.stats.values())
        hit_rate = (self.stats["exact_hits"] / total * 100) if total > 0 else 0
        return {**self.stats, "hit_rate_percent": round(hit_rate, 2)}


class RateLimiter:
    """Sliding window rate limiter."""

    def __init__(self, max_requests: int = RATE_LIMIT_REQUESTS, window_seconds: int = RATE_LIMIT_WINDOW):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.requests: deque = deque()

    def is_allowed(self) -> Tuple[bool, int]:
        now = time.time()
        while self.requests and now - self.requests[0] > self.window_seconds:
            self.requests.popleft()
        if len(self.requests) < self.max_requests:
            self.requests.append(now)
            return True, 0
        wait_time = int(self.window_seconds - (now - self.requests[0])) + 1
        return False, wait_time


class ConversationMemory:
    """Conversation memory with context retention."""

    def __init__(self, max_turns: int = 10):
        self.max_turns = max_turns
        self.history: List[Dict] = []
        self.context_entities: Set[str] = set()

    def add_turn(self, user_query: str, assistant_response: str, metadata: Dict = None):
        self.history.append({
            "user": user_query,
            "assistant": assistant_response,
            "timestamp": datetime.now().isoformat(),
            "metadata": metadata or {}
        })
        if len(self.history) > self.max_turns:
            self.history.pop(0)
        if metadata and "posters" in metadata:
            for p in metadata["posters"]:
                if p.get("title"):
                    self.context_entities.add(p["title"])

    def get_context_summary(self) -> str:
        if not self.history:
            return ""
        recent = self.history[-3:]
        summary = "Recent conversation:\n"
        for turn in recent:
            summary += f"User: {turn['user'][:80]}...\n"
            summary += f"Assistant: {turn['assistant'][:100]}...\n\n"
        if self.context_entities:
            summary += f"Previously discussed: {', '.join(list(self.context_entities)[:5])}"
        return summary

    def clear(self):
        self.history = []
        self.context_entities = set()

print("✅ Caching and memory systems defined")

In [ ]:
# Cell 12: Enhanced Query Classifier

class EnhancedQueryClassifier:
    """Advanced query classifier with pattern matching."""

    QUERY_PATTERNS = {
        "genre_search": {
            "keywords": ["genre", "comedy", "action", "drama", "horror", "thriller", "romance",
                        "documentary", "biography", "adventure", "sci-fi", "animation", "fantasy",
                        "mystery", "crime", "war", "western", "musical", "sport", "family"],
            "patterns": [r"(find|show|get|list).*(comedy|action|drama|horror)", r"\b(genres?)\b"]
        },
        "actor_search": {
            "keywords": ["actor", "actress", "starring", "star", "played by", "cast", "featuring"],
            "patterns": [r"movies?\s+(with|starring|featuring)\s+", r"(actor|actress)\s+\w+"]
        },
        "director_search": {
            "keywords": ["director", "directed", "filmmaker", "made by"],
            "patterns": [r"(directed|director)\s+by?\s*\w+", r"films?\s+by\s+"]
        },
        "rating_search": {
            "keywords": ["rating", "rated", "best", "top", "highest", "score", "imdb"],
            "patterns": [r"(top|best|highest)\s+\d*\s*(rated|movies?)"]
        },
        "year_search": {
            "keywords": ["year", "released", "from 19", "from 20", "decade", "era"],
            "patterns": [r"(in|from|since)\s*(19|20)\d{2}", r"(decade|era|years?)"]
        },
        "comparison": {
            "keywords": ["compare", "versus", "vs", "difference", "better"],
            "patterns": [r"(compare|versus|vs\.?)\s+"]
        },
        "recommendation": {
            "keywords": ["recommend", "suggest", "similar", "like", "should i watch"],
            "patterns": [r"(recommend|suggest)\s+(me\s+)?", r"similar\s+to\s+"]
        },
        "mood_based": {
            "keywords": ["mood", "feeling", "feel like", "happy", "sad", "excited", "scared",
                        "romantic", "thoughtful", "nostalgic", "adventurous", "relaxed", "inspired"],
            "patterns": [r"(mood|feeling|feel\s+like)", r"i('m|\s+am)\s+(happy|sad|excited|scared)"]
        },
        "trivia": {
            "keywords": ["trivia", "fact", "facts", "interesting", "did you know", "fun fact"],
            "patterns": [r"(trivia|facts?)\s+(about|for)"]
        },
        "review_sentiment": {
            "keywords": ["review", "reviews", "critics", "audience", "reception"],
            "patterns": [r"(reviews?|critics?|reception)"]
        },
        "duration_search": {
            "keywords": ["duration", "runtime", "long", "short", "minutes", "length"],
            "patterns": [r"(duration|runtime|length)", r"(short|long)\s+movies?"]
        }
    }

    def classify(self, query: str) -> Tuple[str, float]:
        query_lower = query.lower()
        scores = {}

        for query_type, config in self.QUERY_PATTERNS.items():
            score = 0
            for keyword in config["keywords"]:
                if keyword in query_lower:
                    score += 1
            for pattern in config["patterns"]:
                if re.search(pattern, query_lower):
                    score += 2
            if score > 0:
                scores[query_type] = score

        if not scores:
            return "general_search", 0.5

        best_type = max(scores, key=scores.get)
        confidence = min(scores[best_type] / 5.0, 1.0)
        return best_type, confidence


# Test classifier
classifier = EnhancedQueryClassifier()
test_queries = [
    "Recommend comedy movies",
    "Movies with Tom Hanks",
    "I'm feeling sad, what should I watch?",
    "Trivia about Inception",
]

print("✅ Query Classifier Tests:")
for q in test_queries:
    qtype, conf = classifier.classify(q)
    print(f"   '{q}' -> {qtype} ({conf:.0%} confidence)")

In [ ]:
# Cell 13: Base Agent Class

class BaseAgent(ABC):
    """Abstract base class for all specialized agents."""

    def __init__(self, name: str, description: str, llm, vectorstore, df):
        self.name = name
        self.description = description
        self.llm = llm
        self.vectorstore = vectorstore
        self.df = df
        self.retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    def get_context(self, query: str, k: int = 5) -> List[Document]:
        self.retriever.search_kwargs["k"] = k
        return self.retriever.invoke(query)

    def format_context(self, docs: List[Document]) -> str:
        return "\n\n---\n\n".join([doc.page_content for doc in docs])

    def extract_posters(self, docs: List[Document], limit: int = 5) -> List[Dict]:
        posters = []
        for doc in docs:
            if doc.metadata.get('poster_url'):
                posters.append(doc.metadata)
        return posters[:limit]

    @abstractmethod
    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        pass

    def _create_response(self, answer: str, docs: List[Document], extra_data: Dict = None) -> Dict[str, Any]:
        response = {
            "answer": answer,
            "posters": self.extract_posters(docs),
            "agent": self.name,
            "description": self.description
        }
        if extra_data:
            response.update(extra_data)
        return response

print("✅ BaseAgent class defined")

In [ ]:
# Cell 14: Specialized Agents (10+ Agents)

class GenreRecommendationAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Genre Expert", "Genre-based movie recommendations", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        docs = self.get_context(query, k=8)
        prompt = f"""You are an expert movie curator specializing in film genres.
CONTEXT: {context}\nMOVIE DATABASE:\n{self.format_context(docs)}\n\nUSER QUERY: {query}\n\n
Provide 3-5 genre-specific recommendations with Title, Year, Rating, and why it's great."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class ActorDirectorAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Filmography Expert", "Actor and director career analysis", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        docs = self.get_context(query, k=10)
        prompt = f"""You are a film industry expert with deep knowledge of actors' and directors' careers.
CONTEXT: {context}\nFILMOGRAPHY DATABASE:\n{self.format_context(docs)}\n\nUSER QUERY: {query}\n\n
List notable films with Title, Year, Genre, Rating, and career highlights."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class RatingFilterAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Rating Analyst", "IMDb ratings and critical reception expert", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        high_rated = self.df[self.df['IMDb Rating'] >= 8.0].nlargest(15, 'IMDb Rating')
        docs = self.get_context(query, k=8)
        top_movies = "\n".join([f"- {row['Title']} ({row['Year']}) - IMDb: {row['IMDb Rating']}/10" for _, row in high_rated.head(10).iterrows()])
        prompt = f"""You are a film critic expert.\nTOP RATED:\n{top_movies}\n\nCONTEXT: {self.format_context(docs)}\n\nQUERY: {query}\n\nRecommend highly-rated movies."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class MovieComparisonAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Comparison Analyst", "Movie comparison expert", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        docs = self.get_context(query, k=8)
        prompt = f"""You are a film analyst expert at comparisons.\nCONTEXT: {context}\n\nMOVIE DATABASE:\n{self.format_context(docs)}\n\nQUERY: {query}\n\nCreate a structured comparison."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class MoodBasedAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Mood Curator", "Mood-based movie recommendations", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        query_lower = query.lower()
        detected_mood = "relaxed"
        for mood in MOOD_GENRE_MAPPING.keys():
            if mood in query_lower:
                detected_mood = mood
                break
        mood_genres = MOOD_GENRE_MAPPING.get(detected_mood, ["Drama"])
        genre_pattern = '|'.join(mood_genres)
        mood_movies = self.df[self.df['Genre'].str.contains(genre_pattern, case=False, na=False)]
        top_mood = mood_movies.nlargest(10, 'IMDb Rating')
        mood_list = "\n".join([f"- {row['Title']} ({row['Year']}) - {row['Genre']} - {row['IMDb Rating']}/10" for _, row in top_mood.head(8).iterrows()])
        docs = self.get_context(f"{detected_mood} {' '.join(mood_genres)}", k=8)
        prompt = f"""You are an empathetic movie curator.\n\nDETECTED MOOD: {detected_mood.upper()}\nRECOMMENDED GENRES: {', '.join(mood_genres)}\n\nMOOD-MATCHED MOVIES:\n{mood_list}\n\nQUERY: {query}\n\nRecommend 4-5 movies that match their mood. Be warm and understanding!"""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs, {"detected_mood": detected_mood})


class TriviaAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Trivia Master", "Movie trivia and behind-the-scenes facts", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        docs = self.get_context(query, k=6)
        prompt = f"""You are an entertaining movie trivia expert.\n\nCONTEXT: {context}\nMOVIE DATABASE:\n{self.format_context(docs)}\n\nQUERY: {query}\n\nShare 5-7 fascinating trivia facts. Use "Did you know..." and "Fun fact:" language!"""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class ReviewSentimentAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Review Analyst", "Critical and audience reception analysis", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        docs = self.get_context(query, k=6)
        movie_ratings = [f"{d.metadata.get('title', 'Unknown')}: IMDb {d.metadata.get('rating', 0)}/10, MetaScore {d.metadata.get('metascore', 0)}" for d in docs[:3]]
        prompt = f"""You are a film critic who synthesizes reception.\n\nRATINGS:\n{chr(10).join(movie_ratings)}\n\nDETAILS:\n{self.format_context(docs)}\n\nQUERY: {query}\n\nAnalyze critical reception, audience response, and give a verdict."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class SimilarMoviesAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Similarity Finder", "Finds movies similar to ones you love", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        docs = self.get_context(query, k=12)
        prompt = f"""You are a movie recommendation expert.\nCONTEXT: {context}\n\nSIMILAR MOVIES:\n{self.format_context(docs)}\n\nQUERY: {query}\n\nRecommend 5-6 similar movies. Explain the CONNECTION."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class DurationAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Runtime Advisor", "Finds movies based on available time", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        query_lower = query.lower()
        if any(word in query_lower for word in ["short", "quick", "brief"]):
            duration_filter = self.df[self.df['Duration (minutes)'] <= 100]
            duration_desc = "short (under 100 minutes)"
        elif any(word in query_lower for word in ["long", "epic", "extended"]):
            duration_filter = self.df[self.df['Duration (minutes)'] >= 150]
            duration_desc = "epic length (150+ minutes)"
        else:
            duration_filter = self.df[(self.df['Duration (minutes)'] >= 90) & (self.df['Duration (minutes)'] <= 130)]
            duration_desc = "standard length (90-130 minutes)"
        top_duration = duration_filter.nlargest(10, 'IMDb Rating')
        docs = self.get_context(query, k=6)
        duration_list = "\n".join([f"- {row['Title']} ({row['Year']}) - {row['Duration (minutes)']} min - {row['IMDb Rating']}/10" for _, row in top_duration.head(8).iterrows()])
        prompt = f"""You are a movie guide for time-conscious viewers.\n\nDURATION: {duration_desc}\n\nMATCHING MOVIES:\n{duration_list}\n\nQUERY: {query}\n\nRecommend movies with exact runtime."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class YearEraAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("Era Explorer", "Cinema history across decades", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        year_match = re.search(r'(19|20)\d{2}', query)
        decade_match = re.search(r'(19|20)\d{1}0s', query)
        if decade_match:
            decade_start = int(decade_match.group()[:4])
            era_filter = self.df[(self.df['Year'] >= decade_start) & (self.df['Year'] < decade_start + 10)]
            era_desc = f"the {decade_match.group()}"
        elif year_match:
            year = int(year_match.group())
            era_filter = self.df[(self.df['Year'] >= year - 2) & (self.df['Year'] <= year + 2)]
            era_desc = f"around {year}"
        else:
            era_filter = self.df[self.df['Year'] >= 2020]
            era_desc = "recent years (2020+)"
        top_era = era_filter.nlargest(10, 'IMDb Rating')
        docs = self.get_context(query, k=8)
        era_list = "\n".join([f"- {row['Title']} ({row['Year']}) - {row['Genre']} - {row['IMDb Rating']}/10" for _, row in top_era.head(8).iterrows()])
        prompt = f"""You are a cinema historian.\n\nERA: {era_desc}\n\nTOP MOVIES:\n{era_list}\n\nQUERY: {query}\n\nRecommend the best from this era."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


class GeneralSearchAgent(BaseAgent):
    def __init__(self, llm, vectorstore, df):
        super().__init__("General Assistant", "Versatile movie knowledge assistant", llm, vectorstore, df)

    def invoke(self, query: str, context: str = "") -> Dict[str, Any]:
        docs = self.get_context(query, k=8)
        prompt = f"""You are a knowledgeable movie assistant.\nCONTEXT: {context}\n\nMOVIE DATABASE:\n{self.format_context(docs)}\n\nQUERY: {query}\n\nProvide helpful, engaging recommendations."""
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return self._create_response(response.content, docs)


print("✅ 11 Specialized Agents defined:")
agents_list = ["Genre Expert", "Filmography Expert", "Rating Analyst", "Comparison Analyst",
               "Mood Curator", "Trivia Master", "Review Analyst", "Similarity Finder",
               "Runtime Advisor", "Era Explorer", "General Assistant"]
for i, agent in enumerate(agents_list, 1):
    print(f"   {i}. {agent}")

In [ ]:
# Cell 15: Multi-Agent Orchestrator

class MultiAgentOrchestrator:
    """Orchestrates multiple specialized agents with intelligent routing."""

    def __init__(self, llm, vectorstore, df):
        self.classifier = EnhancedQueryClassifier()
        self.memory = ConversationMemory()
        self.cache = QueryCache()
        self.rate_limiter = RateLimiter()

        # Initialize all agents
        self.agents = {
            "genre_search": GenreRecommendationAgent(llm, vectorstore, df),
            "actor_search": ActorDirectorAgent(llm, vectorstore, df),
            "director_search": ActorDirectorAgent(llm, vectorstore, df),
            "rating_search": RatingFilterAgent(llm, vectorstore, df),
            "year_search": YearEraAgent(llm, vectorstore, df),
            "comparison": MovieComparisonAgent(llm, vectorstore, df),
            "recommendation": SimilarMoviesAgent(llm, vectorstore, df),
            "mood_based": MoodBasedAgent(llm, vectorstore, df),
            "trivia": TriviaAgent(llm, vectorstore, df),
            "review_sentiment": ReviewSentimentAgent(llm, vectorstore, df),
            "duration_search": DurationAgent(llm, vectorstore, df),
            "general_search": GeneralSearchAgent(llm, vectorstore, df),
        }
        self.query_history = []
        self.agent_usage = {}

    def process(self, query: str) -> Dict[str, Any]:
        start_time = time.time()

        # Input validation
        validation = self._validate_input(query)
        if not validation["valid"]:
            return {"answer": validation["message"], "posters": [], "agent": "Input Validator"}

        # Rate limiting
        allowed, wait_time = self.rate_limiter.is_allowed()
        if not allowed:
            return {"answer": f"Too many requests. Please wait {wait_time} seconds.", "posters": [], "agent": "Rate Limiter"}

        # Check cache
        cached = self.cache.get(query)
        if cached:
            cached["from_cache"] = True
            return cached

        # Classify and route
        query_type, confidence = self.classifier.classify(query)
        agent = self.agents.get(query_type, self.agents["general_search"])

        # Get conversation context
        context = self.memory.get_context_summary()

        # Process
        try:
            result = agent.invoke(query, context)
            result["query_type"] = query_type
            result["confidence"] = confidence
            result["processing_time"] = round(time.time() - start_time, 2)
            result["from_cache"] = False

            self.memory.add_turn(query, result["answer"], {"posters": result.get("posters", [])})
            self._update_stats(agent.name, query_type)
            self.cache.set(query, result)

            logger.info(f"Query: type={query_type}, agent={agent.name}, time={result['processing_time']}s")
            return result

        except Exception as e:
            logger.error(f"Error: {str(e)}")
            return {"answer": f"Error processing request: {str(e)}", "posters": [], "agent": "Error Handler"}

    def _validate_input(self, query: str) -> Dict:
        if not query:
            return {"valid": False, "message": "Please enter a question about movies."}
        query = query.strip()
        if len(query) < 3:
            return {"valid": False, "message": "Question too short. Please provide more details."}
        if len(query) > 1000:
            return {"valid": False, "message": "Question too long. Please keep it under 1000 characters."}
        harmful = [r'<script', r'javascript:', r'DROP TABLE', r'DELETE FROM']
        for pattern in harmful:
            if re.search(pattern, query, re.IGNORECASE):
                return {"valid": False, "message": "Invalid input."}
        return {"valid": True, "message": ""}

    def _update_stats(self, agent_name: str, query_type: str):
        self.agent_usage[agent_name] = self.agent_usage.get(agent_name, 0) + 1
        self.query_history.append({"timestamp": datetime.now().isoformat(), "query_type": query_type, "agent": agent_name})

    def get_stats(self) -> Dict:
        return {"total_queries": len(self.query_history), "agent_usage": self.agent_usage, "cache_stats": self.cache.get_stats(), "agents_available": len(self.agents)}

    def clear_history(self):
        self.memory.clear()


# Initialize the orchestrator
print("🚀 Initializing Multi-Agent Orchestrator...")
orchestrator = MultiAgentOrchestrator(llm, vectorstore, df)
print(f"✅ Multi-Agent System Ready with {len(orchestrator.agents)} agents!")

In [ ]:
# Cell 16: Comprehensive Test Suite with Edge Cases

TEST_CASES = {
    "Basic Functionality": [
        ("BF001", "Genre Search", "Recommend some comedy movies"),
        ("BF002", "Actor Search", "Movies with Tom Hanks"),
        ("BF003", "Director Search", "Films by Christopher Nolan"),
        ("BF004", "Rating Filter", "Top rated movies above 8.5"),
        ("BF005", "Year Search", "Best movies from 2020"),
    ],
    "Advanced Agents": [
        ("AA001", "Mood Based", "I'm feeling happy, what should I watch?"),
        ("AA002", "Mood Sad", "I'm feeling sad today"),
        ("AA003", "Trivia", "Trivia about The Dark Knight"),
        ("AA004", "Review", "Reviews for Inception"),
        ("AA005", "Similar", "Movies similar to Interstellar"),
        ("AA006", "Duration Short", "Short movies under 100 minutes"),
        ("AA007", "Duration Long", "Epic long movies over 3 hours"),
        ("AA008", "Era 90s", "Best movies from the 1990s"),
        ("AA009", "Comparison", "Compare Inception and Interstellar"),
    ],
    "Edge Cases - Input Validation": [
        ("EC001", "Empty Input", ""),
        ("EC002", "Too Short", "hi"),
        ("EC003", "Single Character", "a"),
        ("EC004", "Only Spaces", "   "),
        ("EC005", "Very Long Input", "movie " * 200),
    ],
    "Edge Cases - Unusual Queries": [
        ("EC006", "Non-movie Query", "What's the weather today?"),
        ("EC007", "Numbers Only", "12345"),
        ("EC008", "Special Characters", "!@#$%^&*()"),
        ("EC009", "Mixed Language", "Find me peliculas de accion"),
        ("EC010", "Typos", "Recomend comdy moveis"),
    ],
    "Edge Cases - Security": [
        ("EC011", "SQL Injection", "'; DROP TABLE movies; --"),
        ("EC012", "Script Injection", "<script>alert('xss')</script>"),
        ("EC013", "Command Injection", "movie; rm -rf /"),
    ],
    "Edge Cases - Ambiguous Queries": [
        ("EC014", "Vague Query", "good movie"),
        ("EC015", "Multiple Intents", "comedy movies with action and romance from 90s"),
        ("EC016", "Contradictory", "short movies that are very long"),
    ],
    "Cache & Performance": [
        ("CP001", "Repeated Query 1", "Recommend comedy movies"),
        ("CP002", "Repeated Query 2", "Recommend comedy movies"),  # Should hit cache
        ("CP003", "Similar Query", "Suggest comedy films"),  # Test semantic similarity
    ],
}


def run_comprehensive_tests():
    """Run all test cases and report results."""
    print("=" * 70)
    print("🧪 MULTI-AGENT SYSTEM - COMPREHENSIVE TEST SUITE")
    print("=" * 70)
    
    results = {"passed": 0, "failed": 0, "errors": 0}
    detailed_results = []
    
    for category, tests in TEST_CASES.items():
        print(f"\n📋 {category}")
        print("-" * 50)
        
        for test_id, test_name, query in tests:
            try:
                start_time = time.time()
                result = orchestrator.process(query)
                elapsed = round(time.time() - start_time, 2)
                
                # Determine expected behavior and check
                if test_id in ["EC001", "EC004"]:  # Empty/spaces
                    success = "please" in result['answer'].lower() or "enter" in result['answer'].lower()
                elif test_id in ["EC002", "EC003"]:  # Too short
                    success = "short" in result['answer'].lower() or "details" in result['answer'].lower()
                elif test_id == "EC005":  # Too long
                    success = "long" in result['answer'].lower() or "1000" in result['answer'].lower()
                elif test_id in ["EC011", "EC012", "EC013"]:  # Security
                    success = "invalid" in result['answer'].lower() or len(result['answer']) > 20
                elif test_id == "CP002":  # Cache hit
                    success = result.get('from_cache', False) == True
                else:  # Normal queries
                    success = len(result['answer']) > 50 and result.get('agent') != "Error Handler"
                
                status = "✅ PASS" if success else "❌ FAIL"
                results["passed" if success else "failed"] += 1
                
                # Detailed output
                cache_info = "⚡cached" if result.get('from_cache') else f"{elapsed}s"
                print(f"  [{test_id}] {test_name}: {status}")
                print(f"           Agent: {result['agent']} | {cache_info}")
                
                if not success:
                    print(f"           Response: {result['answer'][:80]}...")
                
                detailed_results.append({
                    "id": test_id, "name": test_name, "status": "PASS" if success else "FAIL",
                    "agent": result['agent'], "time": elapsed, "cached": result.get('from_cache', False)
                })
                
            except Exception as e:
                results["errors"] += 1
                print(f"  [{test_id}] {test_name}: ⚠️ ERROR")
                print(f"           {str(e)[:60]}")
                detailed_results.append({"id": test_id, "name": test_name, "status": "ERROR", "error": str(e)})
    
    # Summary
    total = sum(results.values())
    pass_rate = (results["passed"] / total * 100) if total > 0 else 0
    
    print("\n" + "=" * 70)
    print("📊 TEST RESULTS SUMMARY")
    print("=" * 70)
    print(f"  ✅ Passed:  {results['passed']}/{total}")
    print(f"  ❌ Failed:  {results['failed']}/{total}")
    print(f"  ⚠️ Errors:  {results['errors']}/{total}")
    print(f"  📈 Pass Rate: {pass_rate:.1f}%")
    
    # Cache stats
    cache_stats = orchestrator.cache.get_stats()
    print(f"\n  📦 Cache Performance:")
    print(f"     - Exact Hits: {cache_stats['exact_hits']}")
    print(f"     - Misses: {cache_stats['misses']}")
    print(f"     - Hit Rate: {cache_stats['hit_rate_percent']}%")
    
    # Agent usage
    print(f"\n  🤖 Agent Usage:")
    for agent, count in sorted(orchestrator.agent_usage.items(), key=lambda x: x[1], reverse=True):
        print(f"     - {agent}: {count} queries")
    
    print("=" * 70)
    
    return results, detailed_results


# Run the comprehensive test suite
print("🚀 Starting Comprehensive Test Suite...\n")
test_results, detailed = run_comprehensive_tests()

In [ ]:
# Cell 17: Gradio Chat Interface

def chat_with_agents(message: str, history: list) -> str:
    """Main chat function."""
    result = orchestrator.process(message)

    agent_info = f"**🤖 {result['agent']}**"
    cache_info = " ⚡(cached)" if result.get('from_cache') else ""
    time_info = f" | {result.get('processing_time', 'N/A')}s" if not result.get('from_cache') else ""

    response = f"{agent_info}{cache_info}{time_info}\n\n{result['answer']}"
    return response


# Create Gradio Interface (compatible with latest Gradio version)
demo = gr.ChatInterface(
    fn=chat_with_agents,
    title="🎬 IMDb Movie Chatbot - Enhanced Multi-Agent System",
    description="""Powered by **10+ Specialized AI Agents**:
    Genre Expert | Filmography Expert | Rating Analyst | Mood Curator | Trivia Master | Review Analyst | Similarity Finder | Runtime Advisor | Era Explorer

    **Try asking:**
    - "Recommend comedy movies" | "Movies with Leonardo DiCaprio"
    - "I'm feeling sad, what should I watch?" | "Trivia about Inception"
    - "Top rated documentaries" | "Short movies under 100 minutes"
    - "Compare Inception and Interstellar" | "Best movies from the 1990s"
    """,
    examples=[
        "Recommend comedy movies",
        "Movies with Leonardo DiCaprio",
        "I'm feeling happy, what should I watch?",
        "Trivia about The Dark Knight",
        "Top rated documentaries",
        "Compare Inception and Interstellar",
        "Short movies under 100 minutes",
        "Best movies from the 1990s",
        "Movies similar to The Shawshank Redemption",
        "Reviews for Pulp Fiction"
    ]
)

print("✅ Gradio UI Created!")
print("\n🚀 Launching... (check the output below for the public URL)")

In [ ]:
# Cell 18: Launch the Chatbot!

demo.launch(share=True)

---

## 🎉 Congratulations!

Your Enhanced Multi-Agent IMDb Movie Chatbot is now running!

### 🤖 Available Agents:
| Agent | Description | Example Query |
|-------|-------------|---------------|
| **Genre Expert** | Genre-based recommendations | "Recommend comedy movies" |
| **Filmography Expert** | Actor/director careers | "Movies with Tom Hanks" |
| **Rating Analyst** | Rating-based searches | "Top rated documentaries" |
| **Mood Curator** | Mood-based picks | "I'm feeling sad" |
| **Trivia Master** | Fun facts | "Trivia about Inception" |
| **Review Analyst** | Critical reception | "Reviews for Pulp Fiction" |
| **Similarity Finder** | Similar movies | "Movies like The Matrix" |
| **Runtime Advisor** | Duration-based | "Short movies" |
| **Era Explorer** | Decade/year search | "Best of the 90s" |
| **Comparison Analyst** | Movie comparisons | "Compare action vs thriller" |
| **General Assistant** | Versatile queries | "Tell me about Inception" |

---

*Built with LangChain, FAISS, OpenAI GPT-4o-mini, and Gradio*